- make a numpy grid
- grid class - make_grid, check_winner, mave_move, play_game, 
- computer class - subclasses impossible, hard, medium, easy

- recursion method of teking each pos 7 moves, then the next from there, maybe done in a tree method to store info
- need to calc computer win, other player win, and if not opportunity for future win, 

- create parent children structure
- what does the parent need to pass on to the children - grid, score, who is player 1 and who is player_2, current_player, computer, parent, level, space, lines_dict


In [ ]:
Homework1 - 

In [3]:
import numpy as np
import random
from functools import reduce, wraps
from time import sleep, perf_counter

In [4]:
def timeit(func):
    start = perf_counter()
    @wraps(func)
    def wrapper_timer(*args, **kwargs):
        tic = perf_counter()
        value = func(*args, **kwargs)
        toc = perf_counter()
        elapsed_time = toc - tic
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")
        return value
    return wrapper_timer

In [131]:
class Grid:
    def __init__(self): 
        self.player_1 = 'X'
        self.player_2 = 'O'
        self.current_player = self.player_1
        self.opposition = self.player_2
        self.grid = np.array([None] * 42).reshape(6,7)
        self.space = {n: 5 for n in range(7)}
        self.number_grid = np.arange(42).reshape(6,7)
        self.lines_dict = {n: [None] * 4 for n in range(42)}
        self.apply_lines_to_lines_dict()
        self.children = {}
        self.level = 0
        self.score = 0
        self.winner = False
        
#     determines if there is space in the desired column 
    def has_space(self, column):
        return self.space[column] >= 0
    
#     gets the row index for the desired column
    def get_row(self, column):
        return self.space[column]
        
#     inputs the player counter into the next available space in the column, then changes the current player
    def make_move(self, player, column):
        row = self.get_row(column)
        self.grid[row, column] = player
        self.space[column] -= 1
        self.current_player, self.opposition = self.opposition, self.current_player
        
#     searches the lines parameter for a winner (four in a row of anything except None)
    def winner_search(self, lines):
        for line in lines:
            count = 1
            for i in range(1, len(line)):
                if line[i] == line[i - 1]:
                    count += 1
                else:
                    count = 1
                if count == 4:
                    if not line[i]:
                        continue
                    return line[i]
        return False
            
#     searches all lines in a grid for the winner
    def get_winner(self):
        return self.winner_search(self.get_lines(self.grid)[::-1])

#     searches just the lines that intersect at coordinates grid[row,column]
    def get_winner2(self, row, column):
        return self.winner_search(self.get_lines_from_position(row, column))
    
#     score calculated by opportunities to make four in a row of one player minus opportunities for the other player
    def score_search(self, lines):
        scores = [0, 0]
        for lsts in lines:
            for line in lsts:
                sub_lines = [line[x - 4: x] for x in range(4, len(line) + 1)]
                for i in range(2):
                    for sub in sub_lines:
                        if not (([self.player_1, self.player_2][i - 1] in sub) or (set(sub) == {None})):
                            scores[i] += 1
                            break
        return scores[0] - scores[1]

    def get_score(self):
        return self.score_search(self.get_lines(self.grid))
    
    def get_score2(self, row, column):
        return self.score_search([self.get_lines_from_position(row, column)])
                    
    def input_grid(self, grid):
        self.grid = grid
    
#     returns a list of lines (that have a length greater than 4) from the grid - rows, columns, both diagonals
    def get_lines(self, grid):
        return [[list(x) for x in grid], [list(grid[:,c]) for c in range(7)], [list(np.diag(grid, x)) for x in range(-2, 4)],[list(np.diag(np.fliplr(grid), x)) for x in range(-2, 4)]]
    
#     the children are the possible moves from the current grid. The depth is how many times this iterates.
    def make_children(self, depth_limit):   
        if (self.level == depth_limit) or self.winner:
            return
        for col in range(7):
            if self.has_space(col):
                self.children[col] = GridChild(self.grid.copy(), self.space.copy(), self.number_grid, self.lines_dict, self, self.level, self.current_player, self.opposition, depth_limit, self.get_row(col), col, self.score)

#     gets the lines from coordinates[x,y]
    def get_lines_from_position(self, x, y):
        l1, l2, l3, l4 = self.lines_dict[self.number_grid[x,y]]
        lines = []
        lines.append(self.grid[l1,:])
        if isinstance(l2, int):
            lines.append(self.grid[:, l2])
        if isinstance(l3, int):
            lines.append(np.diag(self.grid, l3))
        if isinstance(l4, int):
            lines.append(np.diag(np.fliplr(self.grid), l4))
        return lines

#     helper function to create self.lines_dict - a dictionary with values of the indicies for the lines to be searched 
    def apply_lines_to_lines_dict(self):
        groups = self.get_lines(self.number_grid)
        for i in range(42):
            for r in range(7):
                for ind in range(4):
                    if r == 6:
                        if i in groups[1][r]:
                            self.lines_dict[i][1] = r
                    elif ind < 2:
                        if i in groups[ind][r]:
                            self.lines_dict[i][ind] = r
                    else:
                        if i in groups[ind][r]:
                            self.lines_dict[i][ind] = r - 2  
                        
    def print_grid(self):
        new_grid = self.grid.copy()
        new_grid[new_grid == None] = '_'
        print()
        print(new_grid)

        
        

In [132]:
class GridChild(Grid):
    def __init__(self, grid, space, number_grid, lines_dict, parent, level, current_player, opposition, depth_limit, row, col, score):
        self.player_1 = 'X'
        self.player_2 = 'O'
        self.current_player = current_player
        self.opposition = opposition
        self.grid = grid
        self.space = space
        self.number_grid = number_grid
        self.lines_dict = lines_dict
        self.parent = parent
        self.children = {}
        self.level = level + 1
        # self.carry = 0
        self.last_move = [row, col]
        self.make_move(self.current_player, col)
        self.winner = self.get_winner2(row, col)
        self.score = self.get_score()
        # self.score2 = score + self.get_score2(row, col) + self.carry
        self.make_children(depth_limit)
        
    def make_move(self, player, column):
        # self.carry = - self.get_score2(self.get_row(column), column)
        super().make_move(player, column)
        
# score seems to be faster than score2, but from make_children the score function definitely takes up the most time


In [194]:
class TestGrid(Grid):
    def make_random_grid(self):
        pool = ['X', 'O', None]
        grid = []
        for x in range(6):
            grid.append(random.choices(pool, k = 7))
        return np.array(grid)
    
    def test_play(self):
        self.grid = self.make_random_grid()
        print(self.grid)
        return self.get_winner()
    
    def test_random_till_winner(self):
        order = []
        while True:
            players = ['X', 'O']
            for p in players:
                while True:
                    col = random.randint(0,6)
                    if self.has_space(col):
                        break
                row = self.space[col]
                self.make_move(p, col)
                winner = self.get_winner2(row, col)
                order.append(p + str(col))
                if winner:
                    self.print_grid(self.grid)
                    print()
                    return winner, self.get_score()
    
    def two_player_game(self):
        print('playing game')
        while True:
            computer_moved = False
            if set(self.space.values()) == {-1}:
                print('Non Winner')
                return
            while True:
                turn = input('input 0 - 6 for the column')
                if turn.isdigit() and 0 <= int(turn) < 7 and self.has_space(int(turn)):
                    break
            row = self.get_row(int(turn))
            self.make_move(self.player_1, int(turn))
            self.print_grid()
            if self.get_winner2(row, int(turn)):
                print('You have won')
                return
            
#             computer turn

            self.make_children(2)
            for child in self.children.values():
                if child.winner:
                    self.make_move(self.player_2, child.last_move[1])
                    self.print_grid()
                    print('computer wins')
                    return
            parents = {}
            for child in self.children.values():
                for grandchild in child.children.values():
                    if grandchild.winner:
                        parents[grandchild.parent.last_move[1]] = grandchild.parent
#                         parents.append(grandchild.parent.last_move[1])
            if parents:
                moves = {0,1,2,3,4,5,6}.difference(set(parents.keys()))
                moves = list(filter(self.has_space, list(moves)))
                if moves:
                    new_moves = [self.children[x] for x in moves]
                    new_moves = sorted(new_moves, key = lambda x: x.score)
                    print(new_moves)
                    move = new_moves[0].last_move[1]
                    self.make_move(self.player_2, move)
                    computer_moved = True
# #                     print('move true')
#                     while True:
#                         if self.has_space(move):
#                             print('move worked', move)
#                             break
            if not computer_moved:
                moves = list(filter(self.has_space, [0,1,2,3,4,5,6]))
                turn = random.choice(moves)
                self.make_move(self.player_2, turn)
                computer_moved = True
            if computer_moved:
                self.print_grid()
                self.children = {}   
        return 

In [195]:
print('sid')

sid


In [197]:
#### g1 = TestGrid()
g1.two_player_game()


playing game


input 0 - 6 for the column 3
input 0 - 6 for the column 3
input 0 - 6 for the column 4



[['_' '_' '_' 'O' '_' '_' '_']
 ['_' '_' '_' 'O' '_' '_' '_']
 ['_' '_' '_' 'O' '_' '_' '_']
 ['_' 'X' 'X' 'O' '_' '_' '_']
 ['_' 'X' 'X' 'X' 'X' '_' '_']
 ['O' 'X' 'X' 'X' 'O' 'O' 'O']]
You have won


In [77]:
g1 = TestGrid()
g1.make_children(3)

In [78]:
g1 = TestGrid()
g1.make_children(4)

In [79]:
g1 = TestGrid()
g1.make_children(5)

In [80]:
g1 = TestGrid()
g1.make_children(6)

In [71]:
g1 = TestGrid()
g1.make_children(7)

In [ ]:
g1 = TestGrid()
g1.make_children(8)

In [ ]:
g1 = TestGrid()
g1.make_children(9)

In [ ]:
g1 = TestGrid()
g1.make_children(10)

In [ ]:
g1 = TestGrid()
g1.make_children(11)

In [26]:
for child in g1.children.values():
    for x in child.children.values():
        for y in x.children.values():
            for z in y.children.values():
                for d in z.children.values():
                    if not d.score == d.score2:
                        print('wrong')
                        
print('finished')
    
# both 15.9, 16.1
# score2 10.2
# score1 9.1
# checkpoint wednesday evening score2 doesn't match up with score1. why?

AttributeError: 'GridChild' object has no attribute 'score2'

In [183]:
x = random.randint(0,5)
y = random.randint(0,6)
print(x,y)
g1.get_winner2(x, y)


# ['X', None, None, None, None, 'O', 'X'] X - only x, should be 0 too
# get all lines, for x, for o branch, sub each line into groups of 4, if 4 has set <= 2 including 

3 3


False

In [610]:
g1.make_move('o', 3)
g1.grid

array([[None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, 'o', None, None, None]], dtype=object)

In [611]:
g1.make_move('x', 3)
# g1.get_next_available_index(3)

In [612]:
g1.grid
g1.make_move('o', 3)

In [613]:
g1.grid

array([[None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, 'o', None, None, None],
       [None, None, None, 'x', None, None, None],
       [None, None, None, 'o', None, None, None]], dtype=object)

In [412]:
g2 = Grid()

In [413]:
g2.make_move('X', 3)

In [414]:
score = g2.get_score()

In [415]:
carry = - g2.get_score2(4,3)

In [416]:
carry

-1

In [417]:
g2.make_move('O',3)

In [418]:
score2 = g2.get_score2(4,3) + carry + score

In [421]:
score2

-1

In [420]:
g2.grid

array([[None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, 'O', None, None, None],
       [None, None, None, 'X', None, None, None]], dtype=object)

In [677]:
None in a and len(a) == 2

False

In [19]:
x = np.array([[None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None],
       [None, None, None, 'o', None, None, None],
       [None, None, None, 'x', None, None, None],
       [None, None, None, 'o', None, None, None]], dtype=object)

In [26]:
y = np.arange(16).reshape(4,4)

In [27]:
y[y == 2] = 10

In [29]:

x[x== None] = '_'

In [30]:
x

array([['_', '_', '_', '_', '_', '_', '_'],
       ['_', '_', '_', '_', '_', '_', '_'],
       ['_', '_', '_', '_', '_', '_', '_'],
       ['_', '_', '_', 'o', '_', '_', '_'],
       ['_', '_', '_', 'x', '_', '_', '_'],
       ['_', '_', '_', 'o', '_', '_', '_']], dtype=object)

In [43]:

line = [1,2,3,4]
sub_lines = [line[x - 4: x] for x in range(4, len(line) + 1)]
print(sub_lines)

[[1, 2, 3, 4]]


In [68]:
x, y = 5, 10

In [69]:
[x,y][1] += 20

In [71]:
v = [x,y]

In [72]:
v[1] += 20

In [73]:
v

10

In [86]:
x = input('input anything')

KeyboardInterrupt: Interrupted by user

In [32]:
random.randint(0,6)

0

In [76]:
random.choice(list({1}))

1

False